# ARGOS Ablation Study (Newton-only vs CAG-only vs ARGOS)

This notebook reproduces the qualitative structure of the ablation
experiments:

- Newton-only optimization
- CAG-only optimization
- Full ARGOS (LCO + CAG)


In [ ]:
import os
os.chdir('/content/argos_repo') if os.path.exists('/content/argos_repo') else None


In [ ]:
!pip install -q -r requirements.txt


In [ ]:
import sys, os
if 'src' not in os.listdir('.'):
    print("Warning: run this from the repo root (where `src/` lives).")
sys.path.append(os.path.abspath('src'))


In [ ]:
from argos.hotel_env import HotelEnvironment
from argos.lco import LexicographicTier, LexicographicTierStructure
from argos.cag import CAGFilter
from argos.argos_core import ARGOSOptimizer
import numpy as np


In [ ]:
def revenue_grad(x: np.ndarray) -> np.ndarray:
    g = np.zeros_like(x)
    g[0] = - (50 + 100 * x[3])
    g[3] = - (100 * x[0])
    return g

def newton_step(x: np.ndarray) -> np.ndarray:
    H_inv = np.array([1e-3, 1.0, 1.0, 1e-3])
    return -H_inv * revenue_grad(x)

def contour_step(x: np.ndarray) -> np.ndarray:
    g = revenue_grad(x)
    norm = np.linalg.norm(g) + 1e-8
    return -0.1 * g / norm


In [ ]:
def run_newton_only(T=200):
    env = HotelEnvironment(seed=200)
    x = env.reset()
    history = []
    for t in range(T):
        x = x - 0.05 * newton_step(x)
        x, metrics = env.step(x)
        metrics['epoch'] = t
        metrics['mode'] = 'newton'
        history.append(metrics)
    return history

def run_cag_only(T=200):
    env = HotelEnvironment(seed=200)
    x = env.reset()
    cag = CAGFilter(newton_step_fn=newton_step, contour_step_fn=contour_step)
    history = []
    for t in range(T):
        d = cag.compute_direction(x, revenue_grad)
        x = x - 0.05 * d
        x, metrics = env.step(x)
        metrics['epoch'] = t
        metrics['mode'] = 'cag'
        history.append(metrics)
    return history

def run_argos(T=200):
    env = HotelEnvironment(seed=200)
    x = env.reset()
    tier1 = LexicographicTier(
        name="Feasibility",
        objective=lambda x: 0.0,
        constraints=[lambda x: max(0.0, x[0] - 1.0)],
    )
    tiers = LexicographicTierStructure([tier1])
    cag = CAGFilter(newton_step_fn=newton_step, contour_step_fn=contour_step)
    argos = ARGOSOptimizer(tiers=tiers, cag=cag, step_size=0.05, highest_tier=1)
    history = []
    for t in range(T):
        x = argos.step(x, revenue_grad)
        x, metrics = env.step(x)
        metrics['epoch'] = t
        metrics['mode'] = 'argos'
        history.append(metrics)
    return history


In [ ]:
hist_newton = run_newton_only()
hist_cag = run_cag_only()
hist_argos = run_argos()

import pandas as pd
df = pd.DataFrame(hist_newton + hist_cag + hist_argos)
df.tail()


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(10, 6))
axes = axes.ravel()

for mode, grp in df.groupby('mode'):
    axes[0].plot(grp['epoch'], grp['occupancy'], label=mode)
axes[0].set_title('Occupancy')
axes[0].legend()

for mode, grp in df.groupby('mode'):
    axes[1].plot(grp['epoch'], grp['fatigue'], label=mode)
axes[1].set_title('Fatigue')

for mode, grp in df.groupby('mode'):
    axes[2].plot(grp['epoch'], grp['revpar'], label=mode)
axes[2].set_title('RevPAR')

for mode, grp in df.groupby('mode'):
    axes[3].plot(grp['epoch'], grp['staff'], label=mode)
axes[3].set_title('Staff level')

plt.tight_layout()
plt.show()
